In [12]:
import os
import subprocess

# Clone le repo PER depuis GitHub
# Note: Pour environnement local, utilise le chemin direct. Pour Kaggle/Colab, adapte avec token si privé.

repo_path = "/kaggle/working/PER" if "/kaggle" in os.getcwd() else "C:\\Users\\mathy\\Documents\\SI5\\PER"

if not os.path.exists(repo_path):
    # Pour repos privé, utilise: git clone https://<TOKEN>@github.com/MathysGallay/PER.git
    subprocess.run(["git", "clone", "https://github.com/MathysGallay/PER.git", repo_path], check=True)

os.chdir(repo_path)
print(f"✓ Repo cloné/trouvé: {repo_path}")


✓ Repo cloné/trouvé: C:\Users\mathy\Documents\SI5\PER


In [13]:
import os
import glob
import random
import shutil

# Chemins adaptés (local ou Kaggle)
base_path = "/kaggle/working/PER" if "/kaggle" in os.getcwd() else "."
SOURCE_DIR = os.path.join(base_path, "thermal_coco/augmented")
SOURCE_LABELS = os.path.join(base_path, "mini_coco/labels")
TARGET_DIR = "thermal_yolo"

print(f"Source exists: {os.path.exists(SOURCE_DIR)}")
print(f"Labels exist: {os.path.exists(SOURCE_LABELS)}")

os.makedirs(TARGET_DIR, exist_ok=True)

# YOLO folders
folders = ["images/train", "images/val", "labels/train", "labels/val"]
for f in folders:
    os.makedirs(os.path.join(TARGET_DIR, f), exist_ok=True)

# Récupérer toutes les images
images = glob.glob(os.path.join(SOURCE_DIR, "*.jpg"))
print(f"Images trouvées : {len(images)}")

if len(images) == 0:
    print("ERREUR : Aucune image trouvée!")
else:
    # Train/val split
    train_ratio = 0.8
    random.shuffle(images)
    split = int(len(images) * train_ratio)
    
    train_imgs = images[:split]
    val_imgs = images[split:]
    
    print(f"Train: {len(train_imgs)}, Val: {len(val_imgs)}")
    
    # Copier train
    for img_path in train_imgs:
        img_name = os.path.basename(img_path)
        shutil.copy(img_path, os.path.join(TARGET_DIR, "images/train", img_name))
        
        base_name = img_name.split('_aug')[0] if '_aug' in img_name else img_name.replace('.jpg', '')
        label_src = os.path.join(SOURCE_LABELS, base_name + '.txt')
        label_dst = os.path.join(TARGET_DIR, "labels/train", img_name.replace('.jpg', '.txt'))
        
        if os.path.exists(label_src):
            shutil.copy(label_src, label_dst)
    
    # Copier val
    for img_path in val_imgs:
        img_name = os.path.basename(img_path)
        shutil.copy(img_path, os.path.join(TARGET_DIR, "images/val", img_name))
        
        base_name = img_name.split('_aug')[0] if '_aug' in img_name else img_name.replace('.jpg', '')
        label_src = os.path.join(SOURCE_LABELS, base_name + '.txt')
        label_dst = os.path.join(TARGET_DIR, "labels/val", img_name.replace('.jpg', '.txt'))
        
        if os.path.exists(label_src):
            shutil.copy(label_src, label_dst)
    
    print("✓ Dataset YOLO créé")


Source exists: True
Labels exist: True
Images trouvées : 2500
Train: 2000, Val: 500
✓ Dataset YOLO créé


In [14]:
# Définir les classes animales
ANIMAL_CLASSES = {
    14: "bird",
    15: "cat",
    16: "dog",
    17: "horse",
    18: "sheep",
    19: "cow",
    20: "elephant",
    21: "bear",
    22: "zebra",
    23: "giraffe",
}

classes = list(ANIMAL_CLASSES.values())
print(f"Classes animales : {classes}")

yaml_content = f"""
train: {TARGET_DIR}/images/train
val: {TARGET_DIR}/images/val

nc: {len(classes)}
names: {classes}
"""

with open("data.yaml", "w") as f:
    f.write(yaml_content)

print("✓ data.yaml généré !")


Classes animales : ['bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe']
✓ data.yaml généré !


In [15]:
# Entraînement optimisé - Ultralytics CLI
!yolo train \
    model=yolov8s.pt \
    data=data.yaml \
    imgsz=416 \
    batch=8 \
    epochs=100 \
    patience=10 \
    save_period=10 \
    name=thermal_exp_improved \
    project=runs/train \
    device=0 \
    cache=True \
    exist_ok=True \
    amp=False

New https://pypi.org/project/ultralytics/8.3.240 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.233  Python-3.10.19 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=False, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=thermal_exp_improved, nbs=64, nms=False, o

Traceback (most recent call last):
  File "C:\Users\mathy\anaconda3\envs\PER_GPU\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\mathy\anaconda3\envs\PER_GPU\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\mathy\anaconda3\envs\PER_GPU\Scripts\yolo.exe\__main__.py", line 6, in <module>
  File "C:\Users\mathy\anaconda3\envs\PER_GPU\lib\site-packages\ultralytics\cfg\__init__.py", line 985, in entrypoint
    getattr(model, mode)(**overrides)  # default args from model
  File "C:\Users\mathy\anaconda3\envs\PER_GPU\lib\site-packages\ultralytics\engine\model.py", line 773, in train
    self.trainer.train()
  File "C:\Users\mathy\anaconda3\envs\PER_GPU\lib\site-packages\ultralytics\engine\trainer.py", line 243, in train
    self._do_train()
  File "C:\Users\mathy\anaconda3\envs\PER_GPU\lib\site-packages\ultralytics\engine\trainer.py", line 364, in _do_train
    self._setup_train()
  File "C:\Users

## 📹 Alternative Dataset - Extraction YouTube

Le dataset thermique actuel est artificiel et produit des résultats limités. Une meilleure approche est d'extraire des frames de **vidéos YouTube réelles** (safaris thermiques, documentaires animaliers FLIR, etc.)

**Avantages :**
- Images thermiques authentiques
- Variété de conditions (jour/nuit, météo, distances)
- Qualité caméra professionnelle (FLIR, caméras militaires)

**Sources suggérées :**
- "thermal imaging safari africa"
- "FLIR thermal camera wildlife"
- "infrared animal detection night"

In [16]:
# Installation des dépendances (décommenter si nécessaire)
# !pip install yt-dlp opencv-python-headless

import os
import cv2
import subprocess
from pathlib import Path

def extract_frames_from_youtube(video_url, output_dir="real_thermal_dataset", fps=1, max_frames=500):
    """
    Extrait des frames d'une vidéo YouTube à un FPS donné.
    
    Args:
        video_url: URL YouTube (ex: "https://www.youtube.com/watch?v=...")
        output_dir: Dossier de sortie pour les frames
        fps: Nombre de frames par seconde à extraire (1 = 1 frame/sec, 0.5 = 1 frame/2sec)
        max_frames: Nombre maximal de frames à extraire
    """
    os.makedirs(output_dir, exist_ok=True)
    
    # Télécharger la vidéo avec yt-dlp
    video_path = os.path.join(output_dir, "temp_video.mp4")
    print(f"Téléchargement de la vidéo...")
    try:
        subprocess.run([
            "yt-dlp",
            "-f", "best[height<=720]",  # Max 720p pour économiser bande passante
            "-o", video_path,
            video_url
        ], check=True)
    except FileNotFoundError:
        print("Erreur : yt-dlp n'est pas installé. Lance : pip install yt-dlp")
        return 0
    
    # Extraire les frames
    print(f"🎞️ Extraction des frames (FPS={fps})...")
    cap = cv2.VideoCapture(video_path)
    
    video_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_interval = int(video_fps / fps)
    
    frame_count = 0
    saved_count = 0
    
    while cap.isOpened() and saved_count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        
        if frame_count % frame_interval == 0:
            frame_filename = os.path.join(output_dir, f"frame_{saved_count:05d}.jpg")
            cv2.imwrite(frame_filename, frame, [cv2.IMWRITE_JPEG_QUALITY, 95])
            saved_count += 1
            
            if saved_count % 50 == 0:
                print(f"  ✓ {saved_count} frames extraites...")
        
        frame_count += 1
    
    cap.release()
    os.remove(video_path)  # Supprimer la vidéo temporaire
    
    print(f"✅ Extraction terminée : {saved_count} frames dans {output_dir}")
    return saved_count

# Exemple d'utilisation avec URLs YouTube
# Remplace par des vraies URLs de vidéos thermiques animalières
YOUTUBE_URLS = [
    # "https://www.youtube.com/watch?v=...",  # Safari thermique #1
    # "https://www.youtube.com/watch?v=...",  # Documentaire FLIR wildlife
    # "https://www.youtube.com/watch?v=...",  # Night vision animals
]

# Décommenter pour lancer l'extraction
total_frames = 0
for i, url in enumerate(YOUTUBE_URLS):
    print(f"\n{'='*60}")
    print(f"Vidéo {i+1}/{len(YOUTUBE_URLS)}")
    print(f"{'='*60}")
    frames = extract_frames_from_youtube(
        url, 
        output_dir=f"real_thermal_dataset/video_{i+1}", 
        fps=0.5,  # 1 frame toutes les 2 secondes
        max_frames=300
    )
    total_frames += frames

print(f"\nTotal : {total_frames} frames extraites de {len(YOUTUBE_URLS)} vidéos")
print(f"Dossier : real_thermal_dataset/")

print("Script prêt. Ajoute des URLs YouTube dans YOUTUBE_URLS et décommente le code d'exécution.")
print("Recherches suggérées : 'thermal imaging safari', 'FLIR wildlife night', 'infrared animal detection'")


Total : 0 frames extraites de 0 vidéos
Dossier : real_thermal_dataset/
Script prêt. Ajoute des URLs YouTube dans YOUTUBE_URLS et décommente le code d'exécution.
Recherches suggérées : 'thermal imaging safari', 'FLIR wildlife night', 'infrared animal detection'


## 📹 Alternative Dataset - Extraction YouTube

Le dataset thermique actuel est artificiel et produit des résultats limités. Une meilleure approche est d'extraire des frames de **vidéos YouTube réelles** (safaris thermiques, documentaires animaliers FLIR, etc.)

**Avantages :**
- Images thermiques authentiques
- Variété de conditions (jour/nuit, météo, distances)
- Qualité caméra professionnelle (FLIR, caméras militaires)

**Sources suggérées :**
- "thermal imaging safari africa"
- "FLIR thermal camera wildlife"
- "infrared animal detection night"